# IN PROGRESS

# CockroachDB Serverless Provisioning with Terraform

## Introduction

This notebook guides you through the process of provisioning a CockroachDB Cloud cluster using Terraform. Terraform is an infrastructure-as-code tool that enables you to define and manage resources in a declarative configuration language.

Based on [CockroachDB Terraform Provider documentation](https://www.cockroachlabs.com/docs/cockroachcloud/provision-a-cluster-with-terraform)

### Requirements

Before starting, ensure you have:
- Installed Terraform.
- Created a service account and API key in the CockroachDB Cloud Console with admin privileges or Cluster Creator / Cluster Admin role.


# Set Database Variables

Name your new CRDB cluster:

In [ ]:
ClusterName = "CRDB-Cluster-___"

## Configure regions:
Region options: https://www.cockroachlabs.com/docs/cockroachcloud/regions

*You can create a CockroachDB Serverless cluster with up to six regions. While multi-region CockroachDB Dedicated clusters must have a minimum of three regions, clusters can survive zone failures with only two regions. To survive a regional failure, a minimum of three regions is required. (https://www.cockroachlabs.com/docs/cockroachcloud/plan-your-cluster-serverless)*

*Consider if you will want to use GCP Private Service Connect or AWS PrivateLink for in-network VPC peering i.e. connecting within GCP or AWS instead of over the public internet - to increase security and lower latency (https://www.cockroachlabs.com/docs/cockroachcloud/network-authorization#vpc-peering)*

In [ ]:
CloudHost = "GCP"
GCP-PrimaryRegion = "us-east1"

## Set monthly spend limit:

Understanding resource usage: https://www.cockroachlabs.com/docs/cockroachcloud/serverless-resource-usage#example-request-unit-calculation

In [ ]:
MonthlyCRDB-SpendLimit = 50

## Encryption:

**Secret Management & Encrypted Storage**: Ensure all passwords, secrets, and keys are encrypted at rest and in transit.

Here we'll use HashiCorp's Cloud Vault, a best-in-class enterprise grade secrets manager, with open source code and reputable hosting that interfaces well with multi-cloud architectures.

In [ ]:
#placeholder

**TLS Encryption**: Ensure all connections to CockroachDB are encrypted using TLS. Enforce minimum TLS version 1.2.

In [ ]:
#placeholder

## Step 2: Create a Service Account and API Key

Create a service account and API key in the CockroachDB Cloud Console. Assign admin privilege or Cluster Creator / Cluster Admin role at the organization scope.


# Set up the provisioning script environment

## This script will build and execute the provisioning script environment in your current terminal directory

Navigate to the directory where you want to create your script environment:


In [ ]:
!cd ~/your/sys-admin-scripts/crdb-provisioning

Define a unique conda environment name:

In [ ]:
CondaEnv = "ProvisionCRDB"

Then assign current directory as reference path for provisioning execution:

In [2]:
import os

In [5]:
ProvDir = os.getcwd()
print("Confirm this is where you want your script environment:", ProvDir)

Confirm this is where you want your script environment: /Users/a_/Library/CloudStorage/GoogleDrive-ryan@smithright.com/Shared drives/Smithright DataWorks/SysAdmin_Scripts/cockroachdb_terraform


Create a new conda environment with terraform installed.
*-y auto-approves y/n prompts:

In [ ]:
#needs tested
!conda create --name {CondaEnv} terraform -c conda-forge -y

Initialize & active the new environment:

In [6]:
#needs tested
!conda init
!conda activate {CondaEnv}

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/a_/.bash_profile
No action taken.

CondaError: Run 'conda init' before 'conda activate'



Verify terraform install:

In [7]:
!terraform --version

Terraform v1.8.4
on darwin_arm64


If running in vscode from a new conda env, you may need to install ipykernel:

In [ ]:
!conda install ipykernel -y

# Install CRDB

I'm using brew for a mac install, but you can navigate to find install instructions for other systems from this link:
https://www.cockroachlabs.com/docs/stable/install-cockroachdb-mac.html

In [13]:
!brew install cockroachdb/tap/cockroach

==> Auto-updating Homebrew...
Adjust how often this is run with HOMEBREW_AUTO_UPDATE_SECS or disable with
HOMEBREW_NO_AUTO_UPDATE. Hide these hints with HOMEBREW_NO_ENV_HINTS (see `man brew`).
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
haproxy@2.8                              pedump

You have 13 outdated formulae installed.

==> Tapping cockroachdb/tap
Cloning into '/opt/homebrew/Library/Taps/cockroachdb/homebrew-tap'...
remote: Enumerating objects: 1022, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 1022 (delta 129), reused 180 (delta 116), pack-reused 739
Receiving objects: 100% (1022/1022), 215.72 KiB | 1.52 MiB/s, done.
Resolving deltas: 100% (474/474), done.
Tapped 5 formulae (32 files, 295.9KB).
==> Fetching cockroachdb/tap/cockroach
==> Downloading https://binaries.cockroachdb.com/cockroach-v23.2.5.darwin-11.0-a
################################################################

In [ ]:
! brew update

In [14]:
! brew upgrade cockroach

In [16]:
# check for successful install
! which cockroach

/opt/homebrew/bin/cockroach


## Step 3: Create the Terraform Configuration Files

Terraform uses configuration files to define resources. Create a new directory and download the CockroachDB Serverless `main.tf` example file.


In [11]:
!mkdir -p crdb_terraform 
!curl -o ./crdb_terraform/main.tf https://raw.githubusercontent.com/cockroachdb/terraform-provider-cockroach/main/examples/workflows/cockroach_serverless_cluster/main.tf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2416  100  2416    0     0  14773      0 --:--:-- --:--:-- --:--:-- 14822


Create a `terraform.tfvars` file with the following settings:

In [ ]:
#needs tested
%%writefile terraform.tfvars
cluster_name = "default-cluster"
sql_user_name = "CRDB_SQL"
sql_user_password = "REPLACE_THIS_ZSSZDLFKAEJCRAWEPORCXIWEURAWEAWCERAWERCAWEVRAEW"

Set the environment variable `COCKROACH_API_KEY` with your API key from the CockroachDB Cloud Console:

import os

os.environ['COCKROACH_API_KEY'] = "your_api_key_here"

## Step 4: Provision the Cluster

Initialize the Terraform provider. This reads the `main.tf` configuration file and uses the `terraform.tfvars` file for cluster settings.

In [ ]:
!terraform init -upgrade

Create the Terraform plan to preview the actions that will be taken:

In [ ]:
!terraform plan

Apply the Terraform plan to create the cluster:

In [ ]:
!terraform apply -auto-approve

## Step 5: Get Information About Your Cluster

The `terraform show` command displays detailed information about your cluster resources.

In [ ]:
!terraform show

# Security configuration

## Network Security:

**Virtual Private Cloud (VPC) Peering**: Ensure the CockroachDB cluster is accessible only through VPC peering, thus isolating it from the public internet.

In [ ]:
#placeholder

**Firewall Rules**: Implement strict ingress and egress firewall rules. Only allow traffic from trusted IP addresses or CIDR ranges.


In [ ]:
#placeholder

**Private Endpoints**: Use private endpoints for connecting to your CockroachDB cluster to avoid exposing endpoints publicly.

In [ ]:
#placeholder

## Authentication:

**Service Accounts**: Allow only service accounts for automated access with least privilege.

In [ ]:
#placeholder

**Multi-Factor Authentication (MFA)**: Enforce MFA for accessing the CockroachDB Cloud Console.

In [ ]:
#placeholder

**Rotate Credentials**: Automate rotation of passwords and API keys to minimize risk from compromised credentials.

In [ ]:
#placeholder

# Logging & Monitoring:

**Activity Logging**: Enable detailed logging of all access and modification activities within the CockroachDB cluster.

In [ ]:
#placeholder

**Intrusion Detection Systems (IDS)**: Integrate with IDS to detect and respond to suspicious activities.

In [ ]:
#placeholder

**Automated Alerts**: Set up automated alerts for unusual access patterns or potential breaches.

In [ ]:
#placeholder

# Security Audits & Compliance:

**Vulnerability Scanning**: Conduct regular vulnerability scans to identify and mitigate potential security weaknesses.

In [ ]:
#placeholder

**Compliance Checks**: Ensure that your CockroachDB setup complies with industry standards and regulations.

GDPR configuration:

In [ ]:
#placeholder

HIPAA configuration:

In [ ]:
#placeholder

## Conclusion

You have successfully provisioned a CockroachDB Cloud cluster using Terraform. This notebook covered the setup and execution of Terraform commands to manage CockroachDB resources. For more information, refer to the official [CockroachDB Terraform Provider documentation](https://www.cockroachlabs.com/docs/cockroachcloud/provision-a-cluster-with-terraform).